**Общая информация**

**Срок сдачи:** 21 октября 2019, 08:30 

**Штраф за опоздание:** по 0.5 балла за 24 часа задержки. Через 10 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0919, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Невыполнение PEP8 -0.5 баллов
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [1]:
import numpy as np
import matplotlib.pyplot as plt;
from sklearn import neighbors
import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata, fetch_20newsgroups

from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
%reload_ext pycodestyle_magic

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе. Score не должен уступать значению KNN из sklearn

In [91]:
%%pycodestyle
from scipy.spatial import distance
from scipy.spatial.distance import pdist


class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin,
                             SupervisedIntegerMixin, ClassifierMixin):
    X_test = []
    y_test = []

    def __init__(self, n_neighbors, algorithm='brute'):
        self.n_neighbors = n_neighbors
        self.algorithm = algorithm

    def fit(self, X, y):
        self.X_test = np.array(X, dtype=np.uint8)
        self.y_test = np.array(y, dtype=np.uint8)

    def predict(self, X1):
        self.c = np.array([])
        self.t = np.array([])
        X1 = np.array(X1)
        Res = np.arange(X1.shape[0])
        for f in range(X1.shape[0]):
            X = X1[f]
            X = X[np.newaxis, :]
            N = np.repeat(X, self.X_test.shape[0], axis=0)
            P = self.X_test - N
            if self.algorithm == 'brute':
                D = np.linalg.norm(self.X_test - N, axis=1)
                args = D.argsort()[:self.n_neighbors]
            else:
                self.tree = neighbors.KDTree(self.X_test)
                (D, args) = self.tree.query(X, k=self.n_neighbors)
            args = args.flatten()
            c = np.unique(self.y_test)
            t = np.arange(c.shape[0])
            res = 0
            c_res = 0
            cur_res = 0
            for j in range(c.shape[0]):
                for i in range(self.n_neighbors):
                    if self.y_test[args[i]] == c[j]:
                        cur_res += 1
                if cur_res > res:
                    res = cur_res
                    c_res = j
                t[j] = cur_res
                cur_res = 0
            self.c = np.append(self.c, c, axis=0)
            self.t = np.append(self.t, t, axis=0)
            Res[f] = c[c_res]
        self.c = self.c.reshape(X1.shape[0], -1)
        self.t = self.t.reshape(X1.shape[0], -1)
        return Res

    def predict_proba(self, X):
        self.predict(X)
        return np.array(self.t / np.sum(self.t, axis=1).reshape(-1, 1))

    def score(self, X, y):
        X = np.array(X)
        my_sum = np.count_nonzero(np.subtract(self.predict(X), y))
        return 1 - my_sum / X.shape[0]

65:1: W391 blank line at end of file


**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [47]:
iris = datasets.load_iris()

In [48]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [49]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [50]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [51]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"


**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy

In [52]:
%time clf.fit(X_train, y_train)

CPU times: user 1.07 ms, sys: 0 ns, total: 1.07 ms
Wall time: 1.08 ms


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [53]:
%time my_clf.fit(X_train, y_train)

CPU times: user 268 µs, sys: 7 µs, total: 275 µs
Wall time: 88.2 µs


In [54]:
%time clf.predict(X_test)

CPU times: user 5.73 ms, sys: 42 µs, total: 5.77 ms
Wall time: 5.23 ms


array([1, 2, 1, 1, 0, 0, 2, 0, 0, 1, 2, 2, 0, 2, 1])

In [55]:
%time my_clf.predict(X_test)

CPU times: user 3.39 ms, sys: 4.07 ms, total: 7.46 ms
Wall time: 7.01 ms


array([1, 2, 1, 1, 0, 0, 2, 0, 0, 1, 2, 2, 0, 2, 1])

In [56]:
%time clf.predict_proba(X_test)

CPU times: user 0 ns, sys: 4.8 ms, total: 4.8 ms
Wall time: 7.5 ms


array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [57]:
%time my_clf.predict_proba(X_test)

CPU times: user 2.85 ms, sys: 4.06 ms, total: 6.9 ms
Wall time: 7 ms


array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy. Score не должен уступать значению KNN из sklearn

In [58]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [59]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [60]:
%time clf.fit(X_train, y_train)

CPU times: user 1.79 ms, sys: 47 µs, total: 1.84 ms
Wall time: 1.9 ms


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [61]:
%time my_clf.fit(X_train, y_train)

CPU times: user 72 µs, sys: 2 µs, total: 74 µs
Wall time: 81.3 µs


In [64]:
%time clf.predict(X_test)

CPU times: user 8.39 ms, sys: 0 ns, total: 8.39 ms
Wall time: 12 ms


array([2, 1, 2, 2, 1, 0, 1, 0, 2, 0, 1, 2, 1, 0, 0])

In [65]:
%time my_clf.predict(X_test)

CPU times: user 14.5 ms, sys: 165 µs, total: 14.7 ms
Wall time: 13.6 ms


array([2, 1, 2, 2, 1, 0, 1, 0, 2, 0, 1, 2, 1, 0, 0])

In [70]:
%time clf.predict_proba(X_test)

CPU times: user 0 ns, sys: 3.32 ms, total: 3.32 ms
Wall time: 5.04 ms


array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [71]:
%time my_clf.predict_proba(X_test)

CPU times: user 10.5 ms, sys: 0 ns, total: 10.5 ms
Wall time: 10.6 ms


array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [283]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [3]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])


Переведите во всех документах все буквы в нижний регистр и замените во всех документах символы, не
являющиеся буквами и цифрами, на пробелы. Далее разбейте текста по пробельным символам на токены(термы/слова). Удалите текста, содержащие только пробелы.

In [5]:
 #realize here
# data_tok should be a list of lists of tokens for each line in data.
data = newsgroups['data']
target = newsgroups['target']

data_tok1=[]
data_tok2=[]


for j in range(len(data)):
    for i in range(len(data[j])):
        if not (data[j][i].isalnum() or data[j][i]==' '):
            data[j]=data[j].replace(data[j][i],' ')
    data[j] = data[j].lower()
    k=data[j].split(' ')
    k = [x for x in k if x!='']
    if k!=[]:
        data_tok2.append(k)
    if data[j]!='':
        data_tok1.append(data[j])
    else:
        target = np.delete(target,len(data_tok1))

data_tok=data_tok2



In [60]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower() , map(' '.join, data_tok))), "please make sure that you lowercase the data and drop spaced texts"

Преобразуйте датасет в разреженную матрицу scipy.sparse.csr_matrix, где значение x в позиции (i, j)
означает, что в документе i слово j встретилось x раз

In [61]:
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(dtype=np.int8,min_df=30)
X = vectorizer.fit_transform(data_tok1)

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

In [65]:
from sklearn.model_selection import KFold
from  sklearn.model_selection  import  train_test_split
kf = KFold(n_splits=3)
for train, test in kf.split(X):
    print(train.shape)
    print(test.shape) 

(7397,)
(3699,)
(7397,)
(3699,)
(7398,)
(3698,)


Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальный score в среднем на валидации на 3 фолдах).
Постройте график зависимости среднего score от количества соседей. Можно рассмотреть число соседей от 1 до 10.

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf преобразование( sklearn.feature_extraction.text.TfidfTransformer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим качество(score) вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.